In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('/content/GlobalTemperatures (1).csv')
df.head()

,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty
0,1750-01-01,3.034,3.574,NaN,NaN,NaN,NaN,NaN,NaN
1,1750-02-01,3.083,3.702,NaN,NaN,NaN,NaN,NaN,NaN
2,1750-03-01,5.626,3.076,NaN,NaN,NaN,NaN,NaN,NaN
3,1750-04-01,8.490,2.451,NaN,NaN,NaN,NaN,NaN,NaN
4,1750-05-01,11.573,2.072,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.isnull().sum()
df=df.dropna()
df.isnull().sum()

dt                                           0
LandAverageTemperature                       0
LandAverageTemperatureUncertainty            0
LandMaxTemperature                           0
LandMaxTemperatureUncertainty                0
LandMinTemperature                           0
LandMinTemperatureUncertainty                0
LandAndOceanAverageTemperature               0
LandAndOceanAverageTemperatureUncertainty    0
dtype: int64

In [4]:
#Scaling the dataset
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [ ]:
#splitting dataset into x and y
x=scaler.fit_transform(df[['LandAverageTemperature','LandAverageTemperatureUncertainty','LandMaxTemperature','LandMaxTemperatureUncertainty','LandMinTemperature','LandMinTemperatureUncertainty','LandAndOceanAverageTemperature','LandAndOceanAverageTemperatureUncertainty']])
y=df['LandAverageTemperature']

In [6]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [7]:
#reshaping input data from 2D into 3D
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_train

array([[[ 1.01907376, -0.15029761,  1.05336014, ..., -0.23971883,
          0.80582187,  0.04713811]],

       [[-1.279077  , -0.75304806, -1.30964482, ..., -0.4528543 ,
         -1.10582155, -0.74124469]],

       [[-1.31497461, -0.78430179, -1.35072643, ..., -0.68842507,
         -1.25419963, -1.01310082]],

       ...,

       [[ 0.77365613, -0.16369207,  0.79712229, ..., -0.44163664,
          0.74851183, -0.04801154]],

       [[ 1.24525215, -0.89592224,  1.21675818, ..., -0.78489691,
          1.20856236, -0.98591521]],

       [[-0.54235486, -0.39586261, -0.57574622, ..., -0.45509783,
         -0.53586134, -0.00723312]]])

In [8]:
#Building LSTM model
from keras.models import Sequential
from keras.layers import Dense, LSTM , Bidirectional

model = Sequential()
model.add(Bidirectional(LSTM(512, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))))
model.add(Bidirectional(LSTM(256,dropout=0.4)))
model.add(Dense(1))

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
model.fit(X_train, y_train, epochs=100)


Epoch 1/100
50/50 [==============================] - 15s 89ms/step - loss: 43.0981
Epoch 2/100
50/50 [==============================] - 6s 129ms/step - loss: 5.8187
Epoch 3/100
50/50 [==============================] - 5s 91ms/step - loss: 0.4623
Epoch 4/100
50/50 [==============================] - 4s 89ms/step - loss: 0.2219
Epoch 5/100
50/50 [==============================] - 6s 129ms/step - loss: 0.1547
Epoch 6/100
50/50 [==============================] - 5s 91ms/step - loss: 0.1333
Epoch 7/100
50/50 [==============================] - 5s 103ms/step - loss: 0.1117
Epoch 8/100
50/50 [==============================] - 5s 104ms/step - loss: 0.1002
Epoch 9/100
50/50 [==============================] - 5s 108ms/step - loss: 0.0999
Epoch 10/100
50/50 [==============================] - 7s 138ms/step - loss: 0.0920
Epoch 11/100
50/50 [==============================] - 5s 100ms/step - loss: 0.0829
Epoch 12/100
50/50 [==============================] - 5s 98ms/step - loss: 0.0780
Epoch 13/100
50/

In [9]:
#reshaping input data from 2D to 3D
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [13]:
#Prediction using Test Set
y_pred = model.predict(X_test)

13/13 [==============================] - 1s 35ms/step


In [14]:
#Calculating accuracy metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

In [15]:
print('MAE:', mae)
print('MSE:', mse)
print('RMSE:', rmse)
print('R²:', r2)

MAE: 0.043614865938822425
MSE: 0.0030596765871330383
RMSE: 0.055314343412292605
R²: 0.9998332254224851
